In [227]:
from PIL import Image, ImageDraw
import os
from random import randint, uniform
import numpy as np
import pandas as pd
import copy
from pathlib import Path
import Utils.img_utils as iu

In [ ]:
#Set Parameters

purge = True

cards_location = "Datasets/Images/Cards"
bg_location = "Datasets/Images/Backgrounds"
yolo_location_img = "Datasets/YoloFineTuning/images"
yolo_location_labs = "Datasets/YoloFineTuning/labels"

training_size = 750
val_size = 250

In [229]:
def createDataSet(card_list, bg_list, CARD_DIR, BG_DIR, number):

    data = pd.DataFrame(columns=["img", "bbox"])
    sanity_check = pd.DataFrame(columns=["img", "bbox"])

    for i in range(0, number):
        
        y = iu.randomCard(card_list, CARD_DIR)
        y = iu.adjustBrightness(y)
        y = iu.adjustContrast(y)

        x = iu.randomBackground(bg_list, BG_DIR)
        x = iu.adjustBrightness(x)
        x = iu.adjustContrast(x)

        img, bbox = iu.placeCard(y, iu.addNoise(x, 10), min_size=30) #add noise to BG
        img = iu.addNoise(img, 10) #add noise to combined img

        #Make a copy with the box
        box_img = copy.copy(img)
        box_img = iu.drawBox(box_img, bbox)

        img = img.convert("RGB")
        box_img = box_img.convert("RGB")

        data.loc[len(data)] = img, bbox
        sanity_check.loc[len(sanity_check)] = box_img, bbox

    return data, sanity_check
    
def saveDataSet(yolo_img_dir, yolo_lab_dir, set_type, data):

    for index, row in data.iterrows():
        
        row["img"].save(f"{yolo_img_dir}/{set_type}/{index}.png")

        with open(f"{yolo_lab_dir}/{set_type}/{index}.txt", "w") as f:
        
            box = iu.yoloBbox(row["bbox"], (256, 256))

            f.write(f"0 {box[0]} {box[1]} {box[2]} {box[3]}")

def purgeYoloSet(folder):

    folder = Path(folder)

    for file in folder.iterdir():
        if file.is_file():
            file.unlink()

In [230]:
if purge:
    purgeYoloSet(yolo_location_img + "/train")
    purgeYoloSet(yolo_location_img + "/val")
    purgeYoloSet(yolo_location_labs + "/train")
    purgeYoloSet(yolo_location_labs + "/val")

In [231]:
#training_data, tr_sanity = createDataSet(os.listdir(cards_location), os.listdir(bg_location), cards_location, bg_location , 1)

#display(training_data["img"].iloc[0])
#display(tr_sanity["img"].iloc[0])

##################################################################################################################################

training_data, tr_sanity = createDataSet(os.listdir(cards_location), os.listdir(bg_location), cards_location, bg_location , training_size)

saveDataSet(yolo_location_img, yolo_location_labs, "train", training_data)

##################################################################################################################################

val_data, v_sanity = createDataSet(os.listdir(cards_location), os.listdir(bg_location), cards_location, bg_location , val_size)

saveDataSet(yolo_location_img, yolo_location_labs, "val", val_data)

##################################################################################################################################
 